In [2]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2
import os
from os.path import isfile, join

Convert video to frames

In [35]:
def convert_video_to_frames(fileIn, folderOut, fps):
    if not os.path.exists(folderOut):
        os.mkdir(folderOut)
    cap = cv2.VideoCapture(fileIn)
    currentFrame = 0
    while(True):
    # Capture frame-by-frame
        ret, frame = cap.read()
        name = 'frame' + str(currentFrame) + '.jpg'
        cv2.imwrite(os.path.join(folderOut, name), frame)
        #cv2.imshow('video', frame)
        if cv2.waitKey(fps) & 0xFF == ord('q'):
            break
        # Increment frame counter
        currentFrame += 1
    # Release the capture
    cap.release()
    cv2.destroyAllWindows()        

In [36]:
# Video file
fileIn= 'MVI_5248.MOV'
# Output video file
folderOut = './imgOut/'
# Frames per second
fps = 25
convert_video_to_frames(fileIn, folderOut, fps)

KeyboardInterrupt: 

Convert frames to video

In [38]:
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    # Sort by name
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    # Release the writer
    out.release()

In [39]:
# Image folder
pathIn= './imgOut/'
# Output video file
fileOut = 'video.avi'
# Frames per second
fps = 25
convert_frames_to_video(pathIn, fileOut, fps)

Stream video from file and replace perticular frames.

In [18]:
cap = cv2.VideoCapture('MVI_5248.MOV')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Flip current frame
    #frame = cv2.flip(frame,1)
    if currentFrame > 30 and currentFrame < 200:
        frame = cv2.imread('imgOut/frame'+str(currentFrame)+'.jpg')
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Saves image of the current frame in jpg file
    #name = 'frame' + str(currentFrame) + '.jpg'
    #cv2.imwrite(name, frame)
    cv2.rectangle(frame,(500,450),(500,200),(0,255,0),6)
    # Display the resulting frame
    cv2.imshow('video', frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Increment frame counter
    currentFrame += 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

***Add some AI ***

In [3]:
# Arguments for MobileNetSSD pre-trained model
args = {
    "prototxt": "MobileNetSSD_deploy.prototxt.txt",
    "model": "MobileNetSSD_deploy.caffemodel",
    "confidence": 0.2
}

In [4]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
IGNORE = set([])
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [5]:
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [17]:
# initialize the video stream, allow the cammera sensor to warmup,
# and initialize the FPS counter
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
#time.sleep(1.0)
fps = FPS().start()

[INFO] starting video stream...


In [16]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # grab the frame dimensions and convert it to a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (90, 90)),
        0.007843, (90, 90), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > args["confidence"]:
            # extract the index of the class label from the
            # `detections`
            idx = int(detections[0, 0, i, 1])

            # if the predicted class label is in the set of classes
            # we want to ignore then skip the detection
            if CLASSES[idx] in IGNORE:
                continue

            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # draw the prediction on the frame
            label = "{}: {:.2f}%".format(CLASSES[idx],
                confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(10) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    # update the FPS counter
    fps.update()
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] elapsed time: 26.10
[INFO] approx. FPS: 14.29
